## Particle Swarm Optimizaton 

### Terdapat library yang dapat digunakan untuk PSO
1. Pyswarm -- https://pyswarms.readthedocs.io/en/latest/index.html
2. Optunity -- https://optunity.readthedocs.io/en/latest/index.html

Bisa menulis sendiri kode program sebagai berikut

In [72]:
def pso_opt(function, e= 0.001, lr_reduce = False, minimum = True, l_sol = 0,
           u_sol = 10, n_particles = 10, iteration = 10, n_dim = 1,
           c1 = 1.5, c2 = 1.5, w = 0.5, l_teta = 0.4, u_teta=0.9,
            dif_bound = False, dec_lr = False):
    import numpy as np
    import random #untuk membuat inisialisasi acak posis dan kecepatan    
    
    #inisialisasi acak posisi dan kecepatan
    particles_pos = np.zeros((n_particles, n_dim))
    particles_vel = np.zeros((n_particles, n_dim))
    if dif_bound:
        l_sol = []
        u_sol = []
        for i in range (n_dim):
            print('masukkan nilai batas bawah solusi dimensi ke-', i+1)
            x = input()
            x = float(x)
            l_sol.append(x)
            print('masukkan nilai batas atas solusi dimensi ke-', i+1)
            y = input()
            y = float(y)
            u_sol.append(y)
            for j in range (n_particles):
                particles_pos[j][i] = random.uniform(x,y)
                particles_vel[j][i] = random.uniform(-abs(y-x), abs(y-x))
    
    else:
        for i in range(n_dim):
            for j in range(n_particles):
                particles_pos[j][i] = random.uniform(l_sol, u_sol)
                particles_vel[j][i] = random.uniform(-abs(u_sol-l_sol), abs(u_sol-l_sol))
                
    #Menentukan Local Best dan Global Best awal
    #Local Best
    local_value = []
    local_best = []
    for i in particles_pos:
            value = function(i)
            local_value.append(value)
            local_best.append(i)
    #Global Best
    best_value = max(local_value)
    global_best = local_best[local_value.index(max(local_value))]
    
    #update kecepatan dan posisi partikel
    it = 0
    stop = False
    while (it < iteration):
        if stop:
            break
        else:
            #menerapakan nilai inertia (linear decreasing, Xin et al, 2009). Jaga-jaga saja
            if lr_reduce:
                w = u_teta - ((u_teta - l_teta)/iteration)*(it+1)
            for i in range(len(particles_pos)):
                new_particles_pos = np.zeros((1, n_dim))[0]
                for j in range(len(particles_pos[i])):
                    r1 = random.uniform(0, 1)
                    r2 = random.uniform(0, 1)
                    v_new = w*particles_vel[i][j] + c1*r1*(local_best[i][j] - particles_pos[i][j])+c2*r2*(global_best - particles_pos[i][j])
                    p_new = v_new + particles_pos[i][j]
                    new_particles_pos[j] = p_new[0]
                    particles_vel[i][j] = v_new[0]
            #update Local best
            if minimum:
                if function(new_particles_pos) < function(local_best[i]):
                    local_best[i] = new_particles_pos
                    local_value[i] = function(new_particles_pos)
            else:
                if function(new_particles_pos) > function(local_best[i]):
                    local_best[i] = new_particles_pos
                    local_value[i] = function(new_particles_pos)
            
            #update global best
            if minimum:
                best_value = min(local_value)
                global_best = local_best[local_value.index(min(local_value))]
            else:
                best_value = max(local_value)
                global_best = local_best[local_value.index(max(local_value))]
            
        if best_value < e:
            stop = True
        it += 1
    return global_best, best_value

In [73]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import pandas as pd

In [74]:
data = pd.read_csv('fix 4.csv')

In [75]:
X = data.drop('Lama_Kuliah', axis=1)
y = data['Lama_Kuliah']

In [76]:
from imblearn.over_sampling import SMOTE, ADASYN,BorderlineSMOTE
X_resampled, y_resampled = SMOTE().fit_resample(X, y)

In [77]:
X_resampled = X_resampled.drop(['Jenis_TPA', 'Jenis_Beasiswa', 'Motivasi_Beasiswa'], axis=1)

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.2)

In [79]:
def svc(x):
    if (int(abs(x[0]) < 1)):
        boot = False
    else:
        boot = True
        
    if (int(abs(x[1]) < 4)):
        depth = None
    else:
        depth = int(abs(x[1]))
    
    if (int(abs(x[2])) < 2):
        features = 'auto' 
    else:
        features = int(abs(x[2]))
    
    if (int(abs(x[3])) < 1):
        leaf = 1 
    else:
        leaf = int(abs(x[3]))
    
    if (int(abs(x[4])) < 2):
        split = 2
    else:
        split = int(abs(x[4]))
        
    model = RandomForestClassifier(bootstrap= boot,
                                   max_depth= depth,
                                   max_features= features,
                                   min_samples_leaf= leaf,
                                   min_samples_split= split,
                                   n_estimators= int(abs(x[5])))
    model.fit(X_train, y_train)
    acc = (model.predict(X_test)== y_test).mean()
    return acc

In [80]:
pso_opt(svc, n_dim=6, dif_bound = True, minimum=False, lr_reduce = True)

masukkan nilai batas bawah solusi dimensi ke- 1
0
masukkan nilai batas atas solusi dimensi ke- 1
2
masukkan nilai batas bawah solusi dimensi ke- 2
1
masukkan nilai batas atas solusi dimensi ke- 2
8
masukkan nilai batas bawah solusi dimensi ke- 3
0
masukkan nilai batas atas solusi dimensi ke- 3
4
masukkan nilai batas bawah solusi dimensi ke- 4
1
masukkan nilai batas atas solusi dimensi ke- 4
4
masukkan nilai batas bawah solusi dimensi ke- 5
2
masukkan nilai batas atas solusi dimensi ke- 5
6
masukkan nilai batas bawah solusi dimensi ke- 6
50
masukkan nilai batas atas solusi dimensi ke- 6
300


(array([ 0.67005449,  2.11125325,  1.50591858,  3.18509461,  5.4477398 ,
        89.58306194]),
 0.84375)